# Evaluating results of final models

In [4]:
# !pip install evaluate sentence_transformers jiwer torchmetrics

In [5]:
from enum import Enum

from evaluate import load
from sentence_transformers import SentenceTransformer, util
from torchmetrics.text import TranslationEditRate


class SimilarityMetric(Enum):
    """Enum for string similarity metrics. Each metric must implement the evaluate method."""
    SACREBLEU = 0
    TER = 1
    SEMANTIC_SIMILARITY = 2

    def evaluate(self, references: list[str], predictions: list[str]) -> float:
        """Evaluate the given similarity metric between two corpora.
        Performs simple string cleaning for whitespace and punctuation.
        :param references: list of references (official translations)
        :param predictions: list of candidates (model translations)
        :return: average similarity score when evaluating this specific metric on the corpora
        """
        if self == SimilarityMetric.SACREBLEU:
            # sacrebleu expects a list of references for each candidate
            references = [[ref] for ref in references]
            predictions = [cand for cand in predictions]
            sacrebleu = load("sacrebleu")
            results = sacrebleu.compute(predictions=predictions, references=references)
            return round(results["score"], 1)

        elif self == SimilarityMetric.TER:
            references = [[ref] for ref in references]
            predictions = [cand for cand in predictions]
            ter = TranslationEditRate()
            inverted_score = 1 - ter(predictions, references).item()
            return round(inverted_score * 100, 1)

        else:  # semantic similarity
            similarity_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
            n = len(references)
            score = 0
            for reference, candidate in zip(references, predictions):
                reference_embedding = similarity_model.encode(reference)
                candidate_embedding = similarity_model.encode(candidate)
                cosine_similarity = util.cos_sim(reference_embedding, candidate_embedding)
                score += cosine_similarity[0].item()
            # Return as percentage
            return round(score / n * 100, 1)

In [6]:
import csv
import pandas as pd
import os

results_file = "results.csv"
translations_filepath = "translations/"

with open(results_file, 'w', newline='') as csv_file:
    field_names = ['filename', 'sacrebleu', "`ter`", 'semsim']
    writer = csv.DictWriter(csv_file, fieldnames=field_names)
    if csv_file.tell() == 0:
        writer.writeheader()

    for root, dirs, files in os.walk(translations_filepath):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            df = pd.read_csv(file_path)
            print(f"File: {file_path}")

            test_set = file_path.split('/')[-1]
            line = {'filename': test_set}

            for metric in SimilarityMetric:
                score = metric.evaluate(df["reference"].tolist(), df["translation"].tolist())
                print(f"\t{metric.name}: {score}")

                if metric == SimilarityMetric.SACREBLEU:
                    line['sacrebleu'] = score
                elif metric == SimilarityMetric.TER:
                    line["`ter`"] = score
                else:
                    line['semsim'] = score

            writer.writerow(line)

File: translations/clinspen.csv
	SACREBLEU: 49.7
	TER: 60.4
	SEMANTIC_SIMILARITY: 93.1
File: translations/pubmed-te.csv
	SACREBLEU: 45.2
	TER: 54.8
	SEMANTIC_SIMILARITY: 92.2
File: translations/hpo.csv
	SACREBLEU: 47.3
	TER: 62.9
	SEMANTIC_SIMILARITY: 92.0
File: translations/orphanet-definitions-te.csv
	SACREBLEU: 61.0
	TER: 69.0
	SEMANTIC_SIMILARITY: 95.7
File: translations/khresmoi-te.csv
	SACREBLEU: 47.1
	TER: 61.7
	SEMANTIC_SIMILARITY: 94.7
